In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
from dash import dcc, html, dash_table
import plotly.express as px
from dash import Input, Output
import dash

In [2]:
JS_SCRIPTS = [
    "https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4",
    # "https://jayd719.github.io/staticfiles/themeChanger.js",
]
SYTLE_SHEETS = [
    "https://cdn.jsdelivr.net/npm/daisyui@5",
    "https://cdn.jsdelivr.net/npm/daisyui@5/themes.css",
]

In [3]:
class Model:
    def __init__(self, url, id):
        self.datasource = url
        self.identifier = id

        self.data = self._load_data()

    def _load_data(self):
        try:
            response = get(self.datasource).text
            soup = BeautifulSoup(response, "html.parser")
            datatable_raw = soup.find("table", class_=self.identifier)

            headers = self._get_headers(datatable_raw)
            data = self._get_table_data(datatable_raw)

            dataset = pd.DataFrame(data=data, columns=headers)

            # Change Location to Host, split the city
            dataset["Host"] = dataset["Location"].apply(
                lambda cell: (cell.split(",")[1]).strip()
            )
            dataset = dataset.iloc[:-1, :]

            return dataset.set_index(headers[0])
        except Exception as e:
            print(f"Failed To Fetch Data: Error: {e}")
            return None

    def _get_headers(self, raw_table):
        header_row = raw_table.find("tr")
        if not header_row:
            return []
        headers = [header.text.strip() for header in header_row.find_all(["th", "td"])]
        return headers

    def _get_table_data(self, raw_table):
        datarows = raw_table.find_all("tr")
        datarows.pop(0)
        data = []
        for row in datarows:
            row = [cell.text.strip() for cell in row.find_all(["th", "td"])]
            data.append(row)
        return data

    def get_years(self):
        years = list(self.data.index)
        years.reverse()
        return years

    def get_counts(self, col="Winners"):
        df = self.data[col].value_counts().reset_index()
        df.columns = ["Country", col]
        df = df.sort_values(col, ascending=False)
        return df

    def get_by_year(self, year):
        filtered_df = self.data[self.data.index == year].reset_index()
        filtered_df = filtered_df[["Winners", "Runners-up", "Host"]].T
        filtered_df = filtered_df.reset_index()
        filtered_df.columns = ["Category", "Country"]
        return filtered_df

In [4]:
class View:
    def __init__(
        self,
    ):
        # CONSTANTS
        self.title = "FIFA Soccer World Cup"
        self.subtitle = ""
        self.footerText = ""
        self.years = []

    def create_layout(self, years):
        self.years = years
        self.subtitle = f"Winners and Runner-ups from {min(years)} to {max(years)}"

        return html.Div(
            className="",
            children=[
                self._create_header(),
                self._create_controls(),
                self._create_info_table(),
                dcc.Graph(id="output-map", figure={}),
                self._create_footer(),
            ],
        )

    def _create_info_table(self):
        return html.Div(
            className="w-94 mx-auto",
            children=[
                dash_table.DataTable(id="years-table", data=None),
            ],
        )

    def _create_header(self):
        return html.Div(
            children=[
                html.H1(
                    children=self.title,
                    className="text-4xl font-bold text-blue-800 mb-2",
                ),
                html.Span(
                    id="data-output",
                    className="text-lg text-gray-600",
                    children=self.subtitle,
                ),
                html.Hr(className="my-3"),
            ],
            className="my-10 container mx-auto",
        )

    def _create_footer(self):
        return html.Div(
            className="mt-16 mb-5 text-center text-gray-500",
            children=[html.Span(className="", children=self.footerText)],
        )

    def _create_controls(self):
        return html.Div(
            className="container mx-auto mb-5 grid grid-cols-3 gap-10",
            children=[
                self._create_dropdown_one(),
                self._create_count_control(),
                self._create_years_dropdown(),
            ],
        )

    def _create_dropdown_one(self):
        return dcc.Dropdown(
            id="controls-task-1",
            className="",
            options=["Winners", "Runners-up", "Host"],
            value="Winners",
            clearable=False,
        )

    def _create_years_dropdown(self):
        return dcc.Dropdown(
            id="controls-year",
            className="",
            options=self.years,
            value="",
            placeholder="Select An Year",
        )

    def _create_count_control(self):
        return dcc.Checklist(
            id="count-checkbox",
            className="flex gap-2 border border-gray-300 p-1 rounded text-gray-500",
            options=[{"label": "Show Count", "value": "True"}],
            value=[],
        )

    def _update_common(self, fig, filter):
        title = f"FIFA World Cup {filter}"
        fig.update_layout(
            paper_bgcolor="rgba(0,0,0,0)",
            plot_bgcolor="rgba(0,0,0,0)",
            font=dict(family="Arial", size=15, color="#333333"),
            title={
                "text": title,
                "y": 0.95,
                "x": 0.5,
                "font": dict(size=20, color="#333333"),
            },
            margin=dict(l=2, r=2, t=50, b=2),
            hoverlabel=dict(bgcolor="white", font_size=14, font_family="Arial"),
        )
        return fig

    def create_map(self, df, filter):
        fig = px.choropleth(
            df,
            locations="Country",
            locationmode="country names",
            hover_name="Country",
            hover_data={f"{filter}": True, "Country": True},
            projection="natural earth",
            labels={f"{filter}": f"Number of Times {filter} "},
            height=600,
        )
        fig = self._update_common(fig, filter)
        return fig

    def create_map_with_counts(self, df, filter):
        df[filter] = df[filter].astype(str)
        fig = px.choropleth(
            df,
            locations="Country",
            locationmode="country names",
            color=f"{filter}",
            hover_name="Country",
            hover_data={f"{filter}": True, "Country": True},
            projection="natural earth",
            labels={f"{filter}": f"Number of Times {filter} "},
            height=600,
        )

        fig = self._update_common(fig, filter)
        fig.update_layout(
            coloraxis_colorbar={
                "title": f"{filter} Count",
                "thickness": 15,
                "len": 0.75,
                "x": 0.9,
                "y": 0.5,
                "yanchor": "middle",
                "tickfont": dict(size=13),
                "title_font": dict(size=16),
            },
        )

        return fig

    def create_map_by_year(self, df, year):
        title = f"Winners, Runner-Up and Host For Year {year}"
        fig = px.choropleth(
            df,
            locations="Country",
            locationmode="country names",
            color="Category",
            hover_name="Country",
            projection="natural earth",
            height=600,
        )

        fig = self._update_common(fig, title)
        return fig

In [5]:
class Controller:
    def __init__(self, app, model, view):
        self.app = app
        self.model = model
        self.view = view

        self.view.footerText = "Data sourced from Wikipedia. Last updated: 2025"

        self.app.layout = self.view.create_layout(self.model.get_years())
        self._register_callbacks()

    def _register_callbacks(self):
        """Register all Dash callbacks"""

        @self.app.callback(
            Output("output-map", "figure"),
            Output("years-table", "data"),
            Input("controls-task-1", "value"),
            Input("count-checkbox", "value"),
        )
        def update_map(filter_col, checkbox):
            df = self.model.get_counts(filter_col)

            if len(checkbox):
                fig = self.view.create_map_with_counts(df, filter_col)
            else:
                fig = self.view.create_map(df, filter_col)
            return fig, None

        @self.app.callback(
            Output("output-map", "figure", allow_duplicate=True),
            Output("years-table", "data", allow_duplicate=True),
            Input("controls-year", "value"),
            prevent_initial_call=True,
        )
        def update_map_year(year):
            if not year:
                return {}, None
            df = self.model.get_by_year(str(year))
            fig = self.view.create_map_by_year(df, year)
            return fig, df.to_dict("records")


In [6]:
DATASOURCE_URL = "https://en.wikipedia.org/wiki/List_of_FIFA_World_Cup_finals"
TABLE_ID = "plainrowheaders"


app = dash.Dash(
    __name__,
    external_scripts=JS_SCRIPTS,
    external_stylesheets=SYTLE_SHEETS,
)


model = Model(DATASOURCE_URL, TABLE_ID)
view = View()
Controller(app, model, view)


application = app.server

if __name__ == "__main__":
    app.run(debug=True)